# Data Preprocessing and Analysis

In [18]:
!pip install tensorflow-addons

In [19]:
!pip install sentence-transformers

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import tensorflow_addons as tfa

In [51]:
df = pd.read_csv("/content/BEA 2024 Task Data Extended_new.csv")
df.head()

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,NaN,NaN,NaN,NaN,NaN,D,Gastric mucosa,Text,STEP 1,0.38,123.96
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,NaN,NaN,NaN,NaN,NaN,E,Stage of disease,Text,STEP 3,0.50,77.53
2,12,An 18-year-old primigravid woman comes for her...,Repeat measurement of MSAFP concentration,"Triple screening for MSAFP, serum ÃŽÂ²-hCG, an...",Ultrasonography,Amniocentesis for measurement of ÃŽÂ±-fetoprot...,Amniocentesis for chromosomal analysis,NaN,NaN,NaN,NaN,NaN,C,Ultrasonography,Text,STEP 2,0.58,65.15
3,5,"A 26-year-old man, who is admitted to the hosp...",There is no evidence of a familial coagulation...,Half of their daughters will have a clinically...,The sons of their daughters will be at risk fo...,Their sons will be at risk for a clinically ev...,Their sons and daughters will be at risk for a...,NaN,NaN,NaN,NaN,NaN,C,The sons of their daughters will be at risk fo...,Text,STEP 3,0.56,105.24
4,221,A 60-year-old man had a total thyroidectomy an...,Babinski sign present bilaterally,Chvostek sign,Deviation of the tongue to the left side,A drooping left shoulder,Hyporeflexia,NaN,NaN,NaN,NaN,NaN,B,Chvostek sign,Text,STEP 3,0.22,57.02


In [52]:
df.fillna(0, inplace=True)
df.head(2)

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,0,0,0,0,0,D,Gastric mucosa,Text,STEP 1,0.38,123.96
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,0,0,0,0,0,E,Stage of disease,Text,STEP 3,0.50,77.53


In [53]:
cols = df.columns
cols

Index(['ItemNum', 'ItemStem_Text', 'Answer__A', 'Answer__B', 'Answer__C',
       'Answer__D', 'Answer__E', 'Answer__F', 'Answer__G', 'Answer__H',
       'Answer__I', 'Answer__J', 'Answer_Key', 'Answer_Text', 'ItemType',
       'EXAM', 'Difficulty', 'Response_Time'],
      dtype='object')

In [54]:
df.ItemType.value_counts()

ItemType
Text    595
PIX      72
Name: count, dtype: int64

In [55]:
df.EXAM.value_counts()

EXAM
STEP 1    273
STEP 2    220
STEP 3    174
Name: count, dtype: int64

In [56]:
X_cols = ['ItemNum', 'ItemStem_Text', 'Answer__A', 'Answer__B', 'Answer__C',
          'Answer__D', 'Answer__E', 'Answer__F', 'Answer__G', 'Answer__H',
          'Answer__I', 'Answer__J', 'Answer_Key', 'Answer_Text']
y_1 = df['Difficulty']
y_2 = df['Response_Time']

In [59]:
df[X_cols] = df[X_cols].astype('str')

# Dividing data into train and test set

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[X_cols], df.index, test_size=0.2, random_state=1)

y_train1, y_test1 = y_1[y_train], y_1[y_test]
y_train2, y_test2 = y_2[y_train], y_2[y_test]

X_train1, X_test1 = X_train, X_test
X_train2, X_test2 = X_train, X_test

# Vectorizing input

In [61]:
# reference: https://huggingface.co/pritamdeka/S-PubMedBert-MS-MARCO
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [62]:
def vectorizer(data):
    data.columns = data.columns.str.strip()
    data_vec = []
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        ItemStem_Text_vec = model.encode(row['ItemStem_Text'])
        cnt = 0
        temp_vec = np.zeros(768)
        for option in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']:
            if row['Answer__' + option] != '0':
                cnt += 1
                try:
                    temp_vec += model.encode(row['Answer__' + option])
                except:
                    print(f"Error in {row['Answer__' + option]}")
                    cnt -= 1
        temp_vec /= cnt
        Answer_vec = model.encode(row['Answer_Text'])
        final_vec = ItemStem_Text_vec + temp_vec + Answer_vec
        data_vec.append(final_vec)
    return np.array(data_vec)

X_vec = vectorizer(df[X_cols])
X_vec.shape

100%|██████████| 667/667 [15:05<00:00,  1.36s/it]


(667, 768)

In [63]:
scaler = StandardScaler()
X_vec_scaled = scaler.fit_transform(X_vec)

In [64]:
# LSTM expects the input data to be of shape (samples, timesteps, features)
# Since our data is not a sequence, we reshape it to (samples, 1, features)
X_vec_scaled = X_vec_scaled.reshape((X_vec_scaled.shape[0], 1, X_vec_scaled.shape[1]))

# LSTM using Attention model

In [65]:
def build_lstm_model():
    input_layer = tf.keras.layers.Input(shape=(1, 768))
    lstm_layer1 = layers.LSTM(256, activation='relu', return_sequences=True)(input_layer)
    lstm_layer2 = layers.LSTM(128, activation='relu', return_sequences=True)(lstm_layer1)
    lstm_layer3 = layers.LSTM(64, activation='relu', return_sequences=True)(lstm_layer2)

    # Applying attention mechanism
    attention_output = tfa.layers.MultiHeadAttention(num_heads=4, head_size=64, dropout=0.2)([lstm_layer3, lstm_layer3, lstm_layer3])

    # Adding a pooling layer
    pooling_layer = layers.GlobalAveragePooling1D()(attention_output)

    dense_layer1 = layers.Dense(128, activation='relu')(pooling_layer)
    dense_layer2 = layers.Dense(64, activation='relu')(dense_layer1)

    output_layer = layers.Dense(1)(dense_layer2)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

    return model

# Implement K-Fold Cross-Validation

In [66]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
y1_predict_full = np.zeros(len(X_vec_scaled))
y2_predict_full = np.zeros(len(X_vec_scaled))

# LSTM using Attention model - Predicting Difficulty

In [67]:
rmse_scores_1 = []

for train_index, test_index in kfold.split(X_vec_scaled):
    X_train, X_test = X_vec_scaled[train_index], X_vec_scaled[test_index]
    y_train1, y_test1 = y_1[train_index], y_1[test_index]
    y_train2, y_test2 = y_2[train_index], y_2[test_index]

    lstm_model_1 = build_lstm_model()
    lstm_model_1.fit(X_train, y_train1, epochs=100, verbose=1)

    y1_predict = lstm_model_1.predict(X_test)
    y1_predict_full[test_index] = y1_predict.flatten()
    mse = mean_squared_error(y_test1, y1_predict)
    rmse = np.sqrt(mse)
    rmse_scores_1.append(rmse)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17/17 [==============================] - 6s 56ms/step - loss: 0.1851 - root_mean_squared_error: 0.4303
Epoch 2/100
17/17 [==============================] - 1s 56ms/step - loss: 0.0853 - root_mean_squared_error: 0.2920
Epoch 3/100
17/17 [==============================] - 1s 58ms/step - loss: 0.0632 - root_mean_squared_error: 0.2515
Epoch 4/100
17/17 [==============================] - 1s 57ms/step - loss: 0.0448 - root_mean_squared_error: 0.2116
Epoch 5/100
17/17 [==============================] - 1s 36ms/step - loss: 0.0422 - root_mean_squared_error: 0.2055
Epoch 6/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0370 - root_mean_squared_error: 0.1924
Epoch 7/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0322 - root_mean_squared_error: 0.1794
Epoch 8/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0235 - root_mean_squared_error: 0.1534
Epoch 9/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0189 - root_mean

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17/17 [==============================] - 7s 58ms/step - loss: 0.1995 - root_mean_squared_error: 0.4466
Epoch 2/100
17/17 [==============================] - 1s 44ms/step - loss: 0.0907 - root_mean_squared_error: 0.3011
Epoch 3/100
17/17 [==============================] - 1s 33ms/step - loss: 0.0606 - root_mean_squared_error: 0.2462
Epoch 4/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0440 - root_mean_squared_error: 0.2098
Epoch 5/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0351 - root_mean_squared_error: 0.1873
Epoch 6/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0276 - root_mean_squared_error: 0.1662
Epoch 7/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0275 - root_mean_squared_error: 0.1659
Epoch 8/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0249 - root_mean_squared_error: 0.1577
Epoch 9/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0206 - root_mean

...

17/17 [==============================] - 8s 35ms/step - loss: 0.1834 - root_mean_squared_error: 0.4282
Epoch 2/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0934 - root_mean_squared_error: 0.3056
Epoch 3/100
17/17 [==============================] - 1s 36ms/step - loss: 0.0659 - root_mean_squared_error: 0.2566
Epoch 4/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0458 - root_mean_squared_error: 0.2140
Epoch 5/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0331 - root_mean_squared_error: 0.1819
Epoch 6/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0329 - root_mean_squared_error: 0.1813
Epoch 7/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0230 - root_mean_squared_error: 0.1516
Epoch 8/100
17/17 [==============================] - 1s 36ms/step - loss: 0.0223 - root_mean_squared_error: 0.1493
Epoch 9/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0220 - root_mean

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/100
17/17 [==============================] - 6s 35ms/step - loss: 0.1860 - root_mean_squared_error: 0.4313
Epoch 2/100
17/17 [==============================] - 1s 36ms/step - loss: 0.0895 - root_mean_squared_error: 0.2991
Epoch 3/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0735 - root_mean_squared_error: 0.2711
Epoch 4/100
17/17 [==============================] - 1s 36ms/step - loss: 0.0479 - root_mean_squared_error: 0.2189
Epoch 5/100
17/17 [==============================] - 1s 36ms/step - loss: 0.0339 - root_mean_squared_error: 0.1842
Epoch 6/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0415 - root_mean_squared_error: 0.2038
Epoch 7/100
17/17 [==============================] - 1s 36ms/step - loss: 0.0469 - root_mean_squared_error: 0.2165
Epoch 8/100
17/17 [==============================] - 1s 34ms/step - loss: 0.0259 - root_mean_squared_error: 0.1609
Epoch 9/100
17/17 [==============================] - 1s 37ms/step - loss: 0.0167

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17/17 [==============================] - 5s 34ms/step - loss: 0.2022 - root_mean_squared_error: 0.4496
Epoch 2/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0800 - root_mean_squared_error: 0.2829
Epoch 3/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0546 - root_mean_squared_error: 0.2336
Epoch 4/100
17/17 [==============================] - 1s 37ms/step - loss: 0.0383 - root_mean_squared_error: 0.1957
Epoch 5/100
17/17 [==============================] - 1s 35ms/step - loss: 0.0372 - root_mean_squared_error: 0.1930
Epoch 6/100
17/17 [==============================] - 1s 40ms/step - loss: 0.0266 - root_mean_squared_error: 0.1631
Epoch 7/100
17/17 [==============================] - 1s 60ms/step - loss: 0.0179 - root_mean_squared_error: 0.1338
Epoch 8/100
17/17 [==============================] - 1s 59ms/step - loss: 0.0120 - root_mean_squared_error: 0.1095
Epoch 9/100
17/17 [==============================] - 1s 60ms/step - loss: 0.0095 - root_mean

In [68]:
print("RMSE for Difficulty with K-Fold Cross-Validation:", np.mean(rmse_scores_1))

lstm_model_1.summary()

RMSE for Difficulty with K-Fold Cross-Validation: 0.33606504318766145
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 1, 768)]             0         []                            
                                                                                                  
 lstm_42 (LSTM)              (None, 1, 256)               1049600   ['input_15[0][0]']            
                                                                                                  
 lstm_43 (LSTM)              (None, 1, 128)               197120    ['lstm_42[0][0]']             
                                                                                                  
 lstm_44 (LSTM)              (None, 1, 64)                49408     ['lstm_43[0][0]']             
                     

# LSTM using Attention model - Predicting Response Time

In [69]:
rmse_scores_2 = []

for train_index, test_index in kfold.split(X_vec_scaled):
    X_train, X_test = X_vec_scaled[train_index], X_vec_scaled[test_index]
    y_train2, y_test2 = y_2[train_index], y_2[test_index]

    lstm_model_2 = build_lstm_model()
    lstm_model_2.fit(X_train, y_train2, epochs=100, verbose=1)

    y2_predict = lstm_model_2.predict(X_test)
    y2_predict_full[test_index] = y2_predict.flatten()
    mse = mean_squared_error(y_test2, y2_predict)
    rmse = np.sqrt(mse)
    rmse_scores_2.append(rmse)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17/17 [==============================] - 6s 35ms/step - loss: 8096.5732 - root_mean_squared_error: 89.9810
Epoch 2/100
17/17 [==============================] - 1s 48ms/step - loss: 3450.9541 - root_mean_squared_error: 58.7448
Epoch 3/100
17/17 [==============================] - 1s 53ms/step - loss: 1959.1558 - root_mean_squared_error: 44.2624
Epoch 4/100
17/17 [==============================] - 1s 57ms/step - loss: 1133.7814 - root_mean_squared_error: 33.6717
Epoch 5/100
17/17 [==============================] - 1s 56ms/step - loss: 897.9691 - root_mean_squared_error: 29.9661
Epoch 6/100
17/17 [==============================] - 1s 58ms/step - loss: 759.6564 - root_mean_squared_error: 27.5619
Epoch 7/100
17/17 [==============================] - 1s 44ms/step - loss: 691.6235 - root_mean_squared_error: 26.2987
Epoch 8/100
17/17 [==============================] - 1s 34ms/step - loss: 579.0563 - root_mean_squared_error: 24.0636
Epoch 9/100
17/17 [==============================] - 1s 35ms/ste

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17/17 [==============================] - 7s 57ms/step - loss: 8088.5884 - root_mean_squared_error: 89.9366
Epoch 2/100
17/17 [==============================] - 1s 55ms/step - loss: 3743.0337 - root_mean_squared_error: 61.1803
Epoch 3/100
17/17 [==============================] - 1s 35ms/step - loss: 1859.0348 - root_mean_squared_error: 43.1165
Epoch 4/100
17/17 [==============================] - 1s 34ms/step - loss: 1137.2947 - root_mean_squared_error: 33.7238
Epoch 5/100
17/17 [==============================] - 1s 35ms/step - loss: 868.1624 - root_mean_squared_error: 29.4646
Epoch 6/100
17/17 [==============================] - 1s 33ms/step - loss: 694.3073 - root_mean_squared_error: 26.3497
Epoch 7/100
17/17 [==============================] - 1s 35ms/step - loss: 603.3942 - root_mean_squared_error: 24.5641
Epoch 8/100
17/17 [==============================] - 1s 34ms/step - loss: 708.1833 - root_mean_squared_error: 26.6117
Epoch 9/100
17/17 [==============================] - 1s 35ms/ste

...

Epoch 1/100
17/17 [==============================] - 7s 34ms/step - loss: 7903.6987 - root_mean_squared_error: 88.9027
Epoch 2/100
17/17 [==============================] - 1s 35ms/step - loss: 3489.1477 - root_mean_squared_error: 59.0690
Epoch 3/100
17/17 [==============================] - 1s 36ms/step - loss: 1577.6377 - root_mean_squared_error: 39.7195
Epoch 4/100
17/17 [==============================] - 1s 34ms/step - loss: 982.3167 - root_mean_squared_error: 31.3419
Epoch 5/100
17/17 [==============================] - 1s 36ms/step - loss: 837.1945 - root_mean_squared_error: 28.9343
Epoch 6/100
17/17 [==============================] - 1s 34ms/step - loss: 794.5258 - root_mean_squared_error: 28.1873
Epoch 7/100
17/17 [==============================] - 1s 34ms/step - loss: 649.9978 - root_mean_squared_error: 25.4951
Epoch 8/100
17/17 [==============================] - 1s 35ms/step - loss: 613.1007 - root_mean_squared_error: 24.7609
Epoch 9/100
17/17 [==============================] - 

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17/17 [==============================] - 7s 34ms/step - loss: 8118.3032 - root_mean_squared_error: 90.1016
Epoch 2/100
17/17 [==============================] - 1s 35ms/step - loss: 3311.4482 - root_mean_squared_error: 57.5452
Epoch 3/100
17/17 [==============================] - 1s 37ms/step - loss: 1745.1770 - root_mean_squared_error: 41.7753
Epoch 4/100
17/17 [==============================] - 1s 36ms/step - loss: 1128.1552 - root_mean_squared_error: 33.5880
Epoch 5/100
17/17 [==============================] - 1s 56ms/step - loss: 981.3828 - root_mean_squared_error: 31.3270
Epoch 6/100
17/17 [==============================] - 1s 57ms/step - loss: 724.6031 - root_mean_squared_error: 26.9185
Epoch 7/100
17/17 [==============================] - 1s 57ms/step - loss: 728.2568 - root_mean_squared_error: 26.9862
Epoch 8/100
17/17 [==============================] - 1s 59ms/step - loss: 677.7772 - root_mean_squared_error: 26.0342
Epoch 9/100
17/17 [==============================] - 1s 48ms/ste

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17/17 [==============================] - 6s 58ms/step - loss: 8082.7798 - root_mean_squared_error: 89.9043
Epoch 2/100
17/17 [==============================] - 1s 57ms/step - loss: 4232.4927 - root_mean_squared_error: 65.0576
Epoch 3/100
17/17 [==============================] - 1s 58ms/step - loss: 2544.4448 - root_mean_squared_error: 50.4425
Epoch 4/100
17/17 [==============================] - 1s 60ms/step - loss: 1406.9805 - root_mean_squared_error: 37.5097
Epoch 5/100
17/17 [==============================] - 1s 53ms/step - loss: 988.9019 - root_mean_squared_error: 31.4468
Epoch 6/100
17/17 [==============================] - 1s 35ms/step - loss: 944.6222 - root_mean_squared_error: 30.7347
Epoch 7/100
17/17 [==============================] - 1s 36ms/step - loss: 819.3362 - root_mean_squared_error: 28.6241
Epoch 8/100
17/17 [==============================] - 1s 35ms/step - loss: 737.4792 - root_mean_squared_error: 27.1566
Epoch 9/100
17/17 [==============================] - 1s 34ms/ste

In [70]:
print("RMSE for Response Time with K-Fold Cross-Validation:", np.mean(rmse_scores_2))

lstm_model_2.summary()

RMSE for Response Time with K-Fold Cross-Validation: 36.771521925173964
Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 1, 768)]             0         []                            
                                                                                                  
 lstm_57 (LSTM)              (None, 1, 256)               1049600   ['input_20[0][0]']            
                                                                                                  
 lstm_58 (LSTM)              (None, 1, 128)               197120    ['lstm_57[0][0]']             
                                                                                                  
 lstm_59 (LSTM)              (None, 1, 64)                49408     ['lstm_58[0][0]']             
                   

# Add predicted values

In [39]:
df['Predicted_Difficulty'] = y1_predict_full
df['Predicted_Response_Time'] = y2_predict_full

In [40]:
df

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time,Predicted_Difficulty,Predicted_Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,0,0,0,0,0,D,Gastric mucosa,Text,STEP 1,0.38,123.96,0.651222,66.158882
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,0,0,0,0,0,E,Stage of disease,Text,STEP 3,0.50,77.53,0.360435,80.257034
2,12,An 18-year-old primigravid woman comes for her...,Repeat measurement of MSAFP concentration,"Triple screening for MSAFP, serum ÃŽÂ²-hCG, an...",Ultrasonography,Amniocentesis for measurement of ÃŽÂ±-fetoprot...,Amniocentesis for chromosomal analysis,0,0,0,0,0,C,Ultrasonography,Text,STEP 2,0.58,65.15,0.185094,37.618870
3,5,"A 26-year-old man, who is admitted to the hosp...",There is no evidence of a familial coagulation...,Half of their daughters will have a clinically...,The sons of their daughters will be at risk fo...,Their sons will be at risk for a clinically ev...,Their sons and daughters will be at risk for a...,0,0,0,0,0,C,The sons of their daughters will be at risk fo...,Text,STEP 3,0.56,105.24,0.493353,41.878609
4,221,A 60-year-old man had a total thyroidectomy an...,Babinski sign present bilaterally,Chvostek sign,Deviation of the tongue to the left side,A drooping left shoulder,Hyporeflexia,0,0,0,0,0,B,Chvostek sign,Text,STEP 3,0.22,57.02,0.638396,55.733135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,631,A 63-year-old man with chronic obstructive pul...,Continuous nebulized albuterol therapy,CT scan of the chest without contrast,Intravenous aminophylline therapy,Pulmonary function testing,Pulmonary rehabilitation,Titrated oxygen therapy,0,0,0,0,F,Titrated oxygen therapy,Text,STEP 2,0.38,122.77,0.851156,91.877853
462,457,A 58-year-old man comes to the physician becau...,Cremasteric artery,Pampiniform plexus,Testicular artery,Testicular vein,0,0,0,0,0,0,B,Pampiniform plexus,PIX,STEP 1,0.40,79.81,0.663322,49.128178
463,114,A 25-year-old man is brought to the office by ...,Advise the parents not to influence their son'...,Arrange for evaluation of their son's competency,Enroll their son in a smoking cessation program,Initiate a family meeting to discuss the paren...,Obtain legal advice regarding guardianship,0,0,0,0,0,D,Initiate a family meeting to discuss the paren...,Text,STEP 3,0.30,74.61,0.467525,49.885830
464,27,A 78-year-old woman is admitted to the hospita...,Her eligibility to receive Medicare,Her history of an abdominal operation,Her history of arthritis,Her suitability for home dialysis,Her willingness to move to the city,0,0,0,0,0,D,Her suitability for home dialysis,Text,STEP 3,0.16,74.00,0.746654,57.965687


In [41]:
df.to_csv("Predicted_Difficulty_Response_time_LSTM_with_Attention.csv")